In [582]:
import pandas as pd
import numpy as np
import collections
import os
from datetime import datetime
from tqdm import tqdm
import scipy.sparse
from sklearn.feature_extraction.text import TfidfVectorizer

# Finding Similar Documents

## 1.1 Set up the data

In [561]:
df = pd.read_csv(os.getcwd()+'/bank_transactions.csv')
df.head()

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR)
0,T1,C5841053,10/1/94,F,JAMSHEDPUR,17819.05,2/8/16,143207,25.0
1,T2,C2142763,4/4/57,M,JHAJJAR,2270.69,2/8/16,141858,27999.0
2,T3,C4417068,26/11/96,F,MUMBAI,17874.44,2/8/16,142712,459.0
3,T4,C5342380,14/9/73,F,MUMBAI,866503.21,2/8/16,142714,2060.0
4,T5,C9031234,24/3/88,F,NAVI MUMBAI,6714.43,2/8/16,181156,1762.5


In [562]:
df.isna().sum()

TransactionID                 0
CustomerID                    0
CustomerDOB                3397
CustGender                 1100
CustLocation                151
CustAccountBalance         2369
TransactionDate               0
TransactionTime               0
TransactionAmount (INR)       0
dtype: int64

We have some NaN values, we remove the index associated to them.

In [563]:
df = df.dropna()

In [564]:
df.isna().sum()

TransactionID              0
CustomerID                 0
CustomerDOB                0
CustGender                 0
CustLocation               0
CustAccountBalance         0
TransactionDate            0
TransactionTime            0
TransactionAmount (INR)    0
dtype: int64

In [565]:
df.dtypes

TransactionID               object
CustomerID                  object
CustomerDOB                 object
CustGender                  object
CustLocation                object
CustAccountBalance         float64
TransactionDate             object
TransactionTime              int64
TransactionAmount (INR)    float64
dtype: object

In [566]:
df.shape

(1041614, 9)

In [567]:
df.nunique()

TransactionID              1041614
CustomerID                  879358
CustomerDOB                  17233
CustGender                       3
CustLocation                  9275
CustAccountBalance          160723
TransactionDate                 55
TransactionTime              81855
TransactionAmount (INR)      92391
dtype: int64

We remove the id columns because there are not relevant.

In [568]:
df.drop(["TransactionID", "CustomerID"], axis=1, inplace=True)

In [569]:
df.head()

,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR)
0,10/1/94,F,JAMSHEDPUR,17819.05,2/8/16,143207,25.0
1,4/4/57,M,JHAJJAR,2270.69,2/8/16,141858,27999.0
2,26/11/96,F,MUMBAI,17874.44,2/8/16,142712,459.0
3,14/9/73,F,MUMBAI,866503.21,2/8/16,142714,2060.0
4,24/3/88,F,NAVI MUMBAI,6714.43,2/8/16,181156,1762.5


In [570]:
df['CustGender'].value_counts()

M    760978
F    280635
T         1
Name: CustGender, dtype: int64

We remove the index associated to the T gender because it appears only one time and we don't know its meaning.

In [571]:
df.drop(df[df['CustGender'].isin(['T'])].index,axis=0,inplace=True)

In [572]:
df['CustGender'].unique()

array(['F', 'M'], dtype=object)

set -> concatenation of all the values of the attributes of one index

We get a boolean matrix (the characteristic matrix) representation for all the sets where the columns are the sets (one set for one customer) and the lines are the elements. So, if we have a 1 in column c and row r, we know that element for row r is a member of the set for column c.

In [573]:

def periodOfDay(dt) :
    hour = int(dt.split(':')[0])
    if hour>=7 and hour<=12 :
        return 'morning'
    elif hour>=13 and hour<=18 :
        return 'afternoon'
    elif hour>=19 and hour<=22 :
        return 'evening'
    else :
        return 'night'
        

def preprocessedData(df):
    #new_df = pd.DataFrame(['CustGender', 'CustLocation', 'CustomerAge', 'CustAccountBalance', 'TransactionAmount (INR)', 'TransactionDayPeriod'])

    # All the years of birth date are on two digits instead of 1800. This value doesn't make sense. 
    # So, we remove the index associated to this value.
    df.CustomerDOB = pd.to_datetime(df.CustomerDOB)
    df.drop(df[df.CustomerDOB.dt.year == 1800].index, axis=0, inplace=True)
    # Moreover, the dataset only provides the last two digits of the birth years. And, we have values up to 99. 
    # It means that we can then assume that all the years of birth are in the 1900s.
    # So, we process the datetime years that we got in the 2000s by removing 100 years to them.
    df.loc[df.CustomerDOB.dt.year > 2000, 'CustomerDOB'] = df.loc[df.CustomerDOB.dt.year > 2000, 'CustomerDOB'] - pd.DateOffset(years = 100)

    df.CustGender = df['CustGender'].apply(lambda line: 'male' if line=='M' else 'female')
    
    # We get the age of each customer
    df['CustomerAge'] = (( pd.to_datetime('today') - df.CustomerDOB ) / np.timedelta64(1, 'Y')).round(0)
    df['CustomerAge'] = df['CustomerAge'].astype(str)

    # We get the order of the balance of each customer because it will be time-expensive to compare exact values
    df['CustAccountBalance'] = df['CustAccountBalance'].apply(lambda balance: str(format(int(balance),'.1E')))
    df['CustAccountBalanceOrder'] = df['CustAccountBalance'].apply(lambda balance: balance.split('+')[-1])

    df['TransactionAmount (INR)'] = df['TransactionAmount (INR)'].apply(lambda line: str(int(line)))

    df.TransactionTime = df.TransactionTime.apply(lambda timestamp : datetime.utcfromtimestamp(int(timestamp)).strftime('%H:%M:%S'))

    # We get more general information about the transaction time so it's easier to compare it between users
    df['TransactionDayPeriod'] = df.TransactionTime.apply(periodOfDay)

    # We only keep those attributes because they give a good summary about the profile of each customer
    return df[['CustLocation', 'CustomerAge', 'CustAccountBalanceOrder', 'TransactionDayPeriod']]

In [574]:
def getShingles(df) :
    shingles = []
    for i in (df.index) :
        shingles.append(' '.join(df.loc[i].values))
    return shingles

In [575]:
def getCharacteristicMatrix(shingles) :
    vectorizer = TfidfVectorizer() 
    characteristic_matrix = vectorizer.fit_transform(shingles)
    return characteristic_matrix

In [ ]:
preprocessed_df = preprocessedData(df[:1000])
shingles = getShingles(preprocessed_df)
characteristic_matrix = getCharacteristicMatrix(shingles)

In [578]:
preprocessed_df

,CustLocation,CustomerAge,CustAccountBalanceOrder,TransactionDayPeriod
0,JAMSHEDPUR,28.0,04,afternoon
1,JHAJJAR,66.0,03,afternoon
2,MUMBAI,26.0,04,afternoon
3,MUMBAI,49.0,05,afternoon
4,NAVI MUMBAI,35.0,03,night
...,...,...,...,...
998,KASHIPUR,27.0,00,morning
999,UDAIPUR,35.0,04,morning
1000,AMBALA,42.0,05,morning
1001,WRZESNIA,64.0,05,morning


In [313]:
type(characteristic_matrix)

scipy.sparse.csr.csr_matrix

In [589]:
characteristic_matrix

<937x337 sparse matrix of type '<class 'numpy.float64'>'
	with 3943 stored elements in Compressed Sparse Row format>

## 1.2 Fingerprint Hashing

### Min Hashing

In [580]:

class MinHash :
    
    def __init__(self, n_permutations, n_shingles) :        
        self.n_permutations = n_permutations
        self.permutations = []
        self.signature_matrix = []
        for _ in range (n_permutations) :
            self.permutations.append(self.getPermutation(n_shingles))
    
    # input : characteristic matrix (size : n_customers x n_vocs)
    # output : signature matrix (size :  n_permutations x n_customers)
    def __call__(self, characteristic_matrix) :
        n_customers = characteristic_matrix.shape[0]
        #n_shingles = characteristic_matrix.shape[1]

        reduced_signature_matrix = []
        for i in tqdm(range(self.n_permutations)) :

            permuted_matrix = self.permuteMatrix(self.permutations[i], characteristic_matrix)

            # we get the index of the first non-null element for each customer
            index = []
            for i in range(n_customers) :
                #L = self.characteristic_matrix[i].toarray()[0]
                
                #for permutation in self.permutations :
                 #   j = 0
                  #  while L[list(permutation).index(j)] == 0 :
                   #     j += 1
                    #index.append(j)
                index.append(min(np.nonzero(permuted_matrix[i])[1]))
            reduced_signature_matrix.append(index)
        reduced_signature_matrix = np.array(reduced_signature_matrix, dtype=int)
        if len(self.signature_matrix) == 0 :
            self.signature_matrix = reduced_signature_matrix
        else :
            self.signature_matrix = np.concatenate([self.signature_matrix,reduced_signature_matrix], axis=1)
    
    
    def getPermutation(self, n_shingles) :
        return np.random.permutation(range(0,n_shingles))

    # characteristic_matrix : size n_customers x n_vocs
    def permuteMatrix(self, permutation, characteristic_matrix) :

        permuted_matrix = characteristic_matrix.copy()
        for i, j in enumerate(permutation) :
            permuted_matrix[:,j] = characteristic_matrix.getcol(i)
        
        #self.characteristic_matrix = self.permuted_matrix.copy()
        return permuted_matrix






In [581]:

# Divide the signature matrix by band and put the customers into related bucket for each band
# signature matrix : size  n_permutations x n_customers
# b : number of bands
# r : number of rows in each band
class LSH :
    
    def __init__(self, b, r) :
        self.b = b
        self.r = r
        self.buckets = collections.defaultdict(set)
        
    def __call__(self, signature_matrix) :
        bands = np.array_split(signature_matrix, self.b, axis=0)

        for _,band in enumerate(bands) :
            for c in range (signature_matrix.shape[1]) :
                self.buckets[tuple(band[:,c])].add(c)

    def getSimilarities(self, signature_vector) :
        similar_customers = set()
        
        bands = np.array_split(signature_vector, self.b, axis=0)

        for i,band in enumerate(bands) :
            for c in range (signature_vector.shape[1]) :
                similar_customers = similar_customers.union(self.buckets[list(band[:,i])])
                
        return similar_customers
    
    
    

In [594]:
n_permutations = 10
len_chunks = 50000
# characteristic_matrix : lines (customers), columns n_vocabulary()
minHash = MinHash(n_permutations, characteristic_matrix.shape[1])

signature_matrix = np.array([], dtype=int)
if characteristic_matrix.shape[0] < len_chunks :
    characteristic_matrix_reduced = scipy.sparse.lil_matrix(characteristic_matrix)
    minHash(characteristic_matrix_reduced)
else :
    for i in range ((characteristic_matrix.shape[0]//len_chunks)) :
        #print(i*len_chunks, (i+1)*len_chunks)
        characteristic_matrix_reduced = scipy.sparse.lil_matrix(characteristic_matrix[i*len_chunks:(i+1)*len_chunks])
        minHash(characteristic_matrix_reduced)
    if characteristic_matrix.shape[0]%len_chunks != 0 :
        #print((i+1)*len_chunks)
        characteristic_matrix_reduced = scipy.sparse.lil_matrix(characteristic_matrix[(i+1)*len_chunks:])
        minHash(characteristic_matrix_reduced)

signature_matrix = minHash.signature_matrix


100%|██████████| 10/10 [00:04<00:00,  2.45it/s]


In [595]:
signature_matrix

array([[ 21,  91,  79, ..., 151,  34,  28],
       [ 44,  26,  32, ..., 154, 119,  67],
       [159, 138, 259, ...,  73,  67,  23],
       ...,
       [ 39, 113,  34, ...,  12,  12,  61],
       [ 30,  30,   2, ...,  34,  34,  34],
       [ 27,  27,  22, ...,   4, 113, 113]])

### Threshold

We need to fix a value for the number of permutations that we want to compute.
This value is equal to b*r (b : number of bands and r : number of rows in each band).

And, we know that we can approximately link the threshold theta with r and b in this way $$\theta \approx (\frac{1}{b})^\frac{1}{r}$$ in order to be around the transition point to get the better compromise between false positives and false negatives.

So, we first need to fix theta. But we need to be careful because if we take a value too high of theta we increase the chance of getting false negatives (ie to don't output potential similar customers). And, if we take a theta too low we increase the chance of getting false positives (ie to output a lot of customers that are not similar). 

To find a good balance between false positives and flase negatives it is necessary to be closest of the transition point represented the approximation between theta, r and b.

In [543]:
theta = 0.6

Once theta is fixed, we get all the couples (b,r) that verify the approximation. We try to get the maximum value for b and r while having good computing times.

The values of b and r have also a huge impact on the false positives and false negatives. Indeed, if we decrease r, b increases and we get more false positives because when r is little we have less buckets because we have less keys. Moreover, if we increase r, b decreases and we get more false negatives because we increase the number of keys ie the number of buckets.

In [546]:
B = range(5,80)
R = range(2,20)
epsilon = 0.05

couples = []
for b in B :
    for r in R :
        if (1/b)**(1/r) >= theta-epsilon and (1/b)**(1/r) <= theta+epsilon :
            couples.append((b,r))

couples

[(5, 3),
 (6, 3),
 (6, 4),
 (7, 4),
 (8, 4),
 (9, 4),
 (9, 5),
 (10, 4),
 (10, 5),
 (11, 5),
 (12, 5),
 (13, 5),
 (14, 5),
 (14, 6),
 (15, 5),
 (15, 6),
 (16, 5),
 (16, 6),
 (17, 5),
 (17, 6),
 (18, 5),
 (18, 6),
 (19, 5),
 (19, 6),
 (20, 6),
 (21, 6),
 (21, 7),
 (22, 6),
 (22, 7),
 (23, 6),
 (23, 7),
 (24, 6),
 (24, 7),
 (25, 6),
 (25, 7),
 (26, 6),
 (26, 7),
 (27, 6),
 (27, 7),
 (28, 6),
 (28, 7),
 (29, 6),
 (29, 7),
 (30, 6),
 (30, 7),
 (31, 6),
 (31, 7),
 (32, 6),
 (32, 7),
 (32, 8),
 (33, 6),
 (33, 7),
 (33, 8),
 (34, 6),
 (34, 7),
 (34, 8),
 (35, 6),
 (35, 7),
 (35, 8),
 (36, 6),
 (36, 7),
 (36, 8),
 (37, 7),
 (37, 8),
 (38, 7),
 (38, 8),
 (39, 7),
 (39, 8),
 (40, 7),
 (40, 8),
 (41, 7),
 (41, 8),
 (42, 7),
 (42, 8),
 (43, 7),
 (43, 8),
 (44, 7),
 (44, 8),
 (45, 7),
 (45, 8),
 (46, 7),
 (46, 8),
 (47, 7),
 (47, 8),
 (48, 7),
 (48, 8),
 (49, 7),
 (49, 8),
 (49, 9),
 (50, 7),
 (50, 8),
 (50, 9),
 (51, 7),
 (51, 8),
 (51, 9),
 (52, 7),
 (52, 8),
 (52, 9),
 (53, 7),
 (53, 8),
 (53, 9

Huge computation time, we need to work on chunks

In [596]:
b = 5
r = 2
lsh = LSH(b, r)
lsh(signature_matrix)


In [612]:
lsh.buckets

defaultdict(set,
            {(21, 44): {0,
              51,
              69,
              91,
              140,
              346,
              383,
              484,
              555,
              585,
              591,
              602,
              611,
              677,
              684,
              734,
              748,
              751,
              844,
              845,
              851,
              922},
             (91, 26): {1},
             (79, 32): {2},
             (79, 155): {3, 219, 303, 697, 732},
             (36, 12): {4,
              8,
              49,
              90,
              92,
              98,
              132,
              148,
              149,
              156,
              173,
              180,
              203,
              278,
              279,
              282,
              286,
              376,
              395,
              418,
              427,
              456,
              469,
              4

In [608]:
preprocessed_df.iloc[302]

CustLocation               SONIPAT
CustomerAge                   26.0
CustAccountBalanceOrder         04
TransactionDayPeriod       evening
Name: 322, dtype: object

## 1.3 Locality Sensitive Hashing

In [605]:
query_df = pd.read_csv(os.getcwd()+'/query_users.csv')
query_df.head(10)

,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR)
0,27/7/78,M,DELHI,94695.61,2/9/16,140310,65.0
1,6/11/92,M,PANCHKULA,7584.09,2/9/16,120214,6025.0
2,14/8/91,M,PATNA,7180.60,10/8/16,221732,541.5
3,3/1/87,M,CHENNAI,56847.75,29/8/16,144138,1000.0
4,4/1/95,M,GURGAON,84950.13,25/9/16,233309,80.0
5,10/1/81,M,WORLD TRADE CENTRE BANGALORE,23143.95,11/9/16,192906,303.0
6,20/9/76,F,CHITTOOR,15397.80,28/8/16,92633,20.0
7,10/4/91,M,MOHALI,426.30,2/8/16,203754,50.0
8,19/3/90,M,MOHALI,4609.34,26/8/16,184015,300.0
9,19/12/70,M,SERAMPORE,6695988.46,27/8/16,144030,299.0


In [606]:
query_df = preprocessedData(query_df)
query_df.head()

,CustLocation,CustomerAge,CustAccountBalanceOrder,TransactionDayPeriod
0,DELHI,44.0,04,afternoon
1,PANCHKULA,31.0,03,morning
2,PATNA,31.0,03,afternoon
3,CHENNAI,36.0,04,afternoon
4,GURGAON,28.0,04,afternoon


In [ ]:
# on formate nos données d'entrée
query_df = preprocessedData(query_df)

# on récupère la characteristic matrix associée
shingles = getShingles(query_df)
characteristic_matrix = getCharacteristicMatrix(shingles)

# on récupère les documents similaires pour chaque customer
similar_customers = dict()
for i in range(query_df.shape[0]) :
    similar_customers[i] = lsh.getSimilarities(characteristic_matrix[i])


In [211]:
query_df.shape

(46, 6)

In [ ]:
# Get the k most_similar customer of a query user

def jaccard_similarity(list1, list2):
    intersection = set(list1).intersection(set(list2))
    union = set(list1+list2)
    return len(intersection) / len(union)

def getKMostSimilarCustomers(customerIndex) :
    similarCustomers = similar_customers[customerIndex]
    customerData = query_df.iloc[customerIndex]
    jaccard_similarities = []
    for index in similarCustomers :
        # Ne prendre que les colonnes communes à df et query_df
        similarCustomerData = df.iloc[index].values
        jaccard_similarities.append(jaccard_similarity(customerData, similarCustomerData))
    
    # sort en fonction de la jaccard similarity et récup les k premiers

In [617]:
A = [(0,1),(1,15)]
A = sorted(A, key=lambda x:x[1], reverse=True)
A


[(1, 15), (0, 1)]

# 2. Grouping Customers together !

# Command Line Question

In [2]:
full_df = pd.read_csv(os.getcwd()+'/bank_transactions.csv')
full_df.head()

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR)
0,T1,C5841053,10/1/94,F,JAMSHEDPUR,17819.05,2/8/16,143207,25.0
1,T2,C2142763,4/4/57,M,JHAJJAR,2270.69,2/8/16,141858,27999.0
2,T3,C4417068,26/11/96,F,MUMBAI,17874.44,2/8/16,142712,459.0
3,T4,C5342380,14/9/73,F,MUMBAI,866503.21,2/8/16,142714,2060.0
4,T5,C9031234,24/3/88,F,NAVI MUMBAI,6714.43,2/8/16,181156,1762.5


In [4]:
full_df.to_csv(os.getcwd()+'/bank_transactions_v2.csv', sep=';', index=False)

In [80]:
from collections import Counter

print("Location which has the maximum number of purchases been made :", Counter(full_df.CustLocation).most_common(1))

Location which has the maximum number of purchases been made : [('MUMBAI', 103595)]


In [9]:
data = full_df[full_df.CustGender == 'F']
print("Amount spent by females :",sum(list(data['TransactionAmount (INR)'])))

Amount spent by females : 466810951.4299866


In [10]:
data = full_df[full_df.CustGender == 'M']
print("Amount spent by males :", sum(list(data['TransactionAmount (INR)'])))

Amount spent by males : 1181644838.11998


In [8]:
max_average_amount_transaction = 0

df_group_by_customer_id = full_df.groupby("CustomerID")

for c in tqdm(df_group_by_customer_id.indices.keys()) :
    customer_data = df_group_by_customer_id.get_group(c)
    amount_transactions_mean = customer_data["TransactionAmount (INR)"].mean()
    if amount_transactions_mean > max_average_amount_transaction :
        max_average_amount_transaction = amount_transactions_mean
        customer = c

print("Customer "+customer+" has the highest average transaction amount : "+str(max_average_amount_transaction))

100%|██████████| 884265/884265 [03:29<00:00, 4222.08it/s]

Customer C7319271 has the highest average transaction amount : 1560034.99


# Algorithmic Question 